In [1]:
import cv2
import numpy as np
import face_recognition
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display

# Initialize OpenCV's face cascade
face_cascade = cv2.CascadeClassifier('face.xml')

# Example known faces and their encodings (replace with your known faces)
obama_image = face_recognition.load_image_file("obama.jpg")
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

biden_image = face_recognition.load_image_file("biden.jpg")
biden_face_encoding = face_recognition.face_encodings(biden_image)[0]

kesor_image = face_recognition.load_image_file("kesor.jpg")
kesor_face_encoding = face_recognition.face_encodings(kesor_image)[0]

nary_image = face_recognition.load_image_file("nary.jpeg")
nary_face_encoding = face_recognition.face_encodings(nary_image)[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    obama_face_encoding,
    biden_face_encoding,
    kesor_face_encoding,
    nary_face_encoding
]
known_face_names = [
    "Barack Obama",
    "Joe Biden",
    "Sami Chankesor",
    "Pech Channary"
]

print('Learned encoding for', len(known_face_encodings), 'images.')

# Initialize webcam for capture video
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        print("Failed to capture image from camera.")
        break
    
    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
    # Convert frame to RGB for face recognition
    rgb_frame = frame[:, :, ::-1]
    
    # Convert RGB frame to PIL Image
    pil_image = Image.fromarray(rgb_frame)
    
    # Create a Pillow ImageDraw instance to draw with
    draw = ImageDraw.Draw(pil_image)
    
    # Loop through each detected face
    for (x, y, w, h) in faces:
        # Extract the region of interest (ROI) for face recognition
        face_image_bgr = frame[y:y+h, x:x+w]
        
        # Convert BGR to RGB (face_recognition requires RGB)
        face_image_rgb = cv2.cvtColor(face_image_bgr, cv2.COLOR_BGR2RGB)
        
        # Recognize faces in the RGB face region
        face_encodings = face_recognition.face_encodings(face_image_rgb)
        
        # Default name if no known face
        name = "Unknown"
        
        if len(face_encodings) > 0:
            # Compare faces with known faces
            matches = face_recognition.compare_faces(known_face_encodings, face_encodings[0])
            
            # Initialize as unknown face
            best_match_index = None
            min_distance = 1.0
            
            # Loop through matches to find the best match
            for i in range(len(matches)):
                if matches[i]:
                    face_distance = face_recognition.face_distance([known_face_encodings[i]], face_encodings[0])
                    if face_distance < min_distance:
                        min_distance = face_distance
                        best_match_index = i
            
            # Assign name if a match is found within tolerance
            if best_match_index is not None:
                name = known_face_names[best_match_index]
        
        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        # Draw label with name below the face
        cv2.rectangle(frame, (x, y + h), (x + w, y + h + 35), (255, 0, 0), cv2.FILLED)
        cv2.putText(frame, name, (x + 6, y + h + 26), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
    
    # Display the resulting frame
    cv2.imshow('Video', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()

Learned encoding for 4 images.


KeyboardInterrupt: 